In [ ]:
# default_exp core

# utilities

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
import numpy as np
import re
from fastcore.all import *

In [ ]:
#hide
import fastcore
print('Pandas version:', pd.__version__)
print('Numpy version:', np.__version__)
print('Regular expresions version:', re.__version__)
print('Fastcore version:', fastcore.__version__)

Pandas version: 1.1.3
Numpy version: 1.19.2
Regular expresions version: 2.2.1
Fastcore version: 1.1.0


In [ ]:
titanic = pd.read_csv('../Datasets/titanic.csv')

In [ ]:
#export
def is_default_index(df):
    # Check if the index is the same as the default index. We use the name as a proxy
    check_index = ((df.index == pd.RangeIndex(start=0,stop=df.shape[0], step=1)).all())
    return check_index

In [ ]:
#export
def is_multiindex_row_df(df):
    if isinstance(df, pd.core.frame.DataFrame):
        if isinstance(df.index, pd.core.indexes.multi.MultiIndex):
            return True
    return False

In [ ]:
#export
def is_multiindex_col_df(df):
    if isinstance(df, pd.core.frame.DataFrame):
        if isinstance(df.columns, pd.core.indexes.multi.MultiIndex):
            return True
    return False

# Pandas Dataframe methods

Approach inspired by:
1. Official pandas guide: https://pandas.pydata.org/pandas-docs/stable/development/extending.html
2. Pandas Flavor library

## Dataframe methods

The follwing methods enable users to leverage the pandas data analysis library more productively by automating many of the most common tasks. This avoids the user having to search constatnly for answers.

### remove_indexes
Converts a multi-index dataframe into a more readable flat dataframe

In [ ]:
# export
@pd.api.extensions.register_dataframe_accessor('fdt')
class FastDataDataframeUtilities:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
    
    def remove_indexes(self,axis='all'):
        df = self._obj.copy()
        if is_multiindex_col_df(df) and axis in ['columns','all']:
            df.columns = df.columns.map(lambda x: '_'.join([str(i) for i in x]))
        if ((is_multiindex_row_df(df)) or (is_default_index(df) == False)) and axis in ['index','all']:
            df = df.reset_index()
        return df

/Users/peremolinsayuso/opt/anaconda3/envs/dev_react_jupyterlab/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: registration of accessor <class '__main__.FastDataDataframeUtilities'> under name 'fdt' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  


#### Tests

##### Test the flatten_multiindex on all axis

In [ ]:
titanic_pivot = titanic.pivot_table(
    aggfunc={"Survived" : ["sum","size"]}, 
    index=["Pclass","Parents/Children Aboard"], 
    columns=["Sex","Siblings/Spouses Aboard"])

In [ ]:
titanic_pivot.head()

Survived                                    \
                                   size                                     
Sex                              female                              male   
Siblings/Spouses Aboard               0     1    2    3   4   5   8     0   
Pclass Parents/Children Aboard                                              
1      0                           34.0  29.0  1.0  NaN NaN NaN NaN  75.0   
       1                           10.0   7.0  NaN  NaN NaN NaN NaN   9.0   
       2                            5.0   4.0  2.0  2.0 NaN NaN NaN   4.0   
       4                            NaN   NaN  NaN  NaN NaN NaN NaN   NaN   
2      0                           32.0  12.0  NaN  1.0 NaN NaN NaN  72.0   

                                           ...                             \
                                           ...    sum                       
Sex                                        ... female          male         
Siblings/Spouses Aboard            1    2  ...      4   5   8     0     1   
Pclass Parents/Children Aboard             ...                              
1      0                        22.0  2.0  ...    NaN NaN NaN  25.0  10.0   
       1                         5.0  NaN  ...    NaN NaN NaN   2.0   2.0   
       2                         3.0  NaN  ...    NaN NaN NaN   2.0   3.0   
       4                         1.0  NaN  ...    NaN NaN NaN   NaN   0.0   
2      0                        14.0  3.0  ...    NaN NaN NaN   7.0   1.0   

                                                      
                                                      
Sex                                                   
Siblings/Spouses Aboard           2    3   4   5   8  
Pclass Parents/Children Aboard                        
1      0                        1.0  NaN NaN NaN NaN  
       1                        NaN  NaN NaN NaN NaN  
       2                        NaN  0.0 NaN NaN NaN  
       4                        NaN  NaN NaN NaN NaN  
2      0                        0.0  NaN NaN NaN NaN  

[5 rows x 28 columns]

**Remove row index**

In [ ]:
tianic_pivot_flat_rows = titanic_pivot.fdt.remove_indexes(axis='index')

Assert that the column indexes are flattened correctly and the original remains unchanged

In [ ]:
tianic_pivot_flat_rows.head()

Pclass Parents/Children Aboard Survived             \
                                                           size              
Sex                                                      female              
Siblings/Spouses Aboard                                       0     1    2   
0                            1                       0     34.0  29.0  1.0   
1                            1                       1     10.0   7.0  NaN   
2                            1                       2      5.0   4.0  2.0   
3                            1                       4      NaN   NaN  NaN   
4                            2                       0     32.0  12.0  NaN   

                                                ...                       \
                                                ...    sum                 
Sex                                       male  ... female          male   
Siblings/Spouses Aboard    3   4   5   8     0  ...      4   5   8     0   
0                        NaN NaN NaN NaN  75.0  ...    NaN NaN NaN  25.0   
1                        NaN NaN NaN NaN   9.0  ...    NaN NaN NaN   2.0   
2                        2.0 NaN NaN NaN   4.0  ...    NaN NaN NaN   2.0   
3                        NaN NaN NaN NaN   NaN  ...    NaN NaN NaN   NaN   
4                        1.0 NaN NaN NaN  72.0  ...    NaN NaN NaN   7.0   

                                                     
                                                     
Sex                                                  
Siblings/Spouses Aboard     1    2    3   4   5   8  
0                        10.0  1.0  NaN NaN NaN NaN  
1                         2.0  NaN  NaN NaN NaN NaN  
2                         3.0  NaN  0.0 NaN NaN NaN  
3                         0.0  NaN  NaN NaN NaN NaN  
4                         1.0  0.0  NaN NaN NaN NaN  

[5 rows x 30 columns]

In [ ]:
assert is_multiindex_row_df(tianic_pivot_flat_rows) == False

In [ ]:
assert is_multiindex_col_df(tianic_pivot_flat_rows) == True

**Remove col index**

In [ ]:
tianic_pivot_flat_cols = titanic_pivot.fdt.remove_indexes(axis='columns')

In [ ]:
tianic_pivot_flat_cols.head()

Survived_size_female_0  \
Pclass Parents/Children Aboard                           
1      0                                          34.0   
       1                                          10.0   
       2                                           5.0   
       4                                           NaN   
2      0                                          32.0   

                                Survived_size_female_1  \
Pclass Parents/Children Aboard                           
1      0                                          29.0   
       1                                           7.0   
       2                                           4.0   
       4                                           NaN   
2      0                                          12.0   

                                Survived_size_female_2  \
Pclass Parents/Children Aboard                           
1      0                                           1.0   
       1                                           NaN   
       2                                           2.0   
       4                                           NaN   
2      0                                           NaN   

                                Survived_size_female_3  \
Pclass Parents/Children Aboard                           
1      0                                           NaN   
       1                                           NaN   
       2                                           2.0   
       4                                           NaN   
2      0                                           1.0   

                                Survived_size_female_4  \
Pclass Parents/Children Aboard                           
1      0                                           NaN   
       1                                           NaN   
       2                                           NaN   
       4                                           NaN   
2      0                                           NaN   

                                Survived_size_female_5  \
Pclass Parents/Children Aboard                           
1      0                                           NaN   
       1                                           NaN   
       2                                           NaN   
       4                                           NaN   
2      0                                           NaN   

                                Survived_size_female_8  Survived_size_male_0  \
Pclass Parents/Children Aboard                                                 
1      0                                           NaN                  75.0   
       1                                           NaN                   9.0   
       2                                           NaN                   4.0   
       4                                           NaN                   NaN   
2      0                                           NaN                  72.0   

                                Survived_size_male_1  Survived_size_male_2  \
Pclass Parents/Children Aboard                                               
1      0                                        22.0                   2.0   
       1                                         5.0                   NaN   
       2                                         3.0                   NaN   
       4                                         1.0                   NaN   
2      0                                        14.0                   3.0   

                                ...  Survived_sum_female_4  \
Pclass Parents/Children Aboard  ...                          
1      0                        ...                    NaN   
       1                        ...                    NaN   
       2                        ...                    NaN   
       4                        ...                    NaN   
2      0                        ...                    NaN   

                                Survived_sum_fe

In [ ]:
assert is_multiindex_row_df(tianic_pivot_flat_cols) == True

In [ ]:
assert is_multiindex_col_df(tianic_pivot_flat_cols) == False

**Remove both**

In [ ]:
titanic_pivot_flat = titanic_pivot.fdt.remove_indexes(axis='all')

In [ ]:
titanic_pivot_flat

,Pclass,Parents/Children Aboard,Survived_size_female_0,Survived_size_female_1,Survived_size_female_2,Survived_size_female_3,Survived_size_female_4,Survived_size_female_5,Survived_size_female_8,Survived_size_male_0,...,Survived_sum_female_4,Survived_sum_female_5,Survived_sum_female_8,Survived_sum_male_0,Survived_sum_male_1,Survived_sum_male_2,Survived_sum_male_3,Survived_sum_male_4,Survived_sum_male_5,Survived_sum_male_8
0,1,0,34.0,29.0,1.0,NaN,NaN,NaN,NaN,75.0,...,NaN,NaN,NaN,25.0,10.0,1.0,NaN,NaN,NaN,NaN
1,1,1,10.0,7.0,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
2,1,2,5.0,4.0,2.0,2.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,2.0,3.0,NaN,0.0,NaN,NaN,NaN
3,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2,0,32.0,12.0,NaN,1.0,NaN,NaN,NaN,72.0,...,NaN,NaN,NaN,7.0,1.0,0.0,NaN,NaN,NaN,NaN
5,2,1,7.0,9.0,2.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,0.0,6.0,1.0,NaN,NaN,NaN,NaN
6,2,2,5.0,6.0,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN
7,2,3,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,0,60.0,22.0,2.0,1.0,NaN,NaN,NaN,260.0,...,NaN,NaN,NaN,32.0,3.0,1.0,NaN,NaN,NaN,NaN
9,3,1,7.0,10.0,3.0,5.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,2.0,6.0,NaN,0.0,0.0,NaN,NaN


In [ ]:
assert is_multiindex_row_df(titanic_pivot_flat) == False

In [ ]:
assert is_multiindex_col_df(titanic_pivot_flat) == False

### Expand column with several elements
Converts a column with multiple elements separated by a character into several columns

In [ ]:
# export
@patch_to(FastDataDataframeUtilities)
def split_list_to_columns(self, column, separator=',', list_marker='na', split_type='unique'):
    df = self._obj.copy()
    
    type_of_first_not_null_element = type(df[column][df[column].notnull()][0]) 
    
    # First check if it is already a list or it needs pre-processing
    if(type_of_first_not_null_element != list):
        # If not, let's start processing it
        # First we process the surrounding brackets, if they exist
        if list_marker != 'na':
            if list_marker == 'square_brackets':
                df[column] = df[column].str.replace(r"[\[\]']","")
            elif list_marker == 'parentheses':
                df[column] = df[column].str.replace(r'([()])','')
        # Then we process the separator only if we take the unique        
        if split_type == 'unique':
            df[column] = df[column].str.split(separator)
    
    if split_type == 'unique':
        exploded = pd.get_dummies(df[column].apply(pd.Series).stack(dropna=False)).sum(level=0)
        if '' in exploded.columns:
            exploded = exploded.rename(columns={'':'blank'})
        minus_pivoted = df.drop(column,axis=1)
        result = pd.concat([minus_pivoted,exploded], axis=1)
        result = result.fillna(0)
    elif split_type == 'order':
        exploded = df[column].str.split(separator, expand=True)
        minus_pivoted = df.drop(column,axis=1)
        result = pd.concat([minus_pivoted,exploded], axis=1)
    
    result.columns = result.columns.map(str)
    return result

#### Tests

In [ ]:
with_brackets = pd.DataFrame({'A': [[1, 2, 3], [1,8],[], [3, 4]], 'B': [1,2,3,4],'C': [4,3,2,1]})
with_brackets

,A,B,C
0,"[1, 2, 3]",1,4
1,"[1, 8]",2,3
2,[],3,2
3,"[3, 4]",4,1


In [ ]:
assert with_brackets.fdt.split_list_to_columns(
    separator=",", 
    list_marker="square_brackets", 
    column="A", split_type='unique').columns.tolist() == ['B', 'C', '1.0', '2.0', '3.0', '4.0', '8.0']

In [ ]:
without_brackets = pd.DataFrame({'A': ['1,2,3', '1,8' , '', '3,4'], 'B': [1,2,3,4],'C': [4,3,2,1]})
without_brackets

,A,B,C
0,"1,2,3",1,4
1,"1,8",2,3
2,,3,2
3,"3,4",4,1


In [ ]:
assert without_brackets.fdt.split_list_to_columns(
    separator=",", 
    list_marker="na", 
    column="A",split_type='unique').columns.tolist() == ['B', 'C', 'blank', '1', '2', '3', '4', '8']

In [ ]:
without_brackets.fdt.split_list_to_columns(
    separator=",", 
    list_marker="square_brackets", 
    column="A", split_type='order')

,B,C,0,1,2
0,1,4,1,2,3
1,2,3,1,8,None
2,3,2,,None,None
3,4,1,3,4,None


In [ ]:
assert without_brackets.fdt.split_list_to_columns(
    separator=",", 
    list_marker="square_brackets", 
    column="A", split_type='order').iloc[:,4].isna().tolist() == [False, True, True, True]

In [ ]:
strings_and_square_brackets = pd.DataFrame({'A': ['[1,2,3]', '[1,8]' , '[]', '[3,4]'], 'B': [1,2,3,4],'C': [4,3,2,1]})
strings_and_square_brackets

,A,B,C
0,"[1,2,3]",1,4
1,"[1,8]",2,3
2,[],3,2
3,"[3,4]",4,1


In [ ]:
assert strings_and_square_brackets.fdt.split_list_to_columns(
    separator=",", 
    list_marker="square_brackets", 
    column="A", split_type='unique').columns.tolist() == ['B', 'C', 'blank', '1', '2', '3', '4', '8']

In [ ]:
strings_and_round_brackets = pd.DataFrame({'A': ['(1,2,3)', '(1,8)' , '()', '(3,4)'], 'B': [1,2,3,4],'C': [4,3,2,1]})
strings_and_round_brackets

,A,B,C
0,"(1,2,3)",1,4
1,"(1,8)",2,3
2,(),3,2
3,"(3,4)",4,1


In [ ]:
assert strings_and_round_brackets.fdt.split_list_to_columns(
    separator=",", 
    list_marker="parentheses", 
    column="A", split_type='unique').columns.tolist() == ['B', 'C', 'blank', '1', '2', '3', '4', '8']

### Pivot flat
Pivot with the option of flattening

In [ ]:
# export
@patch_to(FastDataDataframeUtilities)
def pivot_table(self, index_type, **kwargs):
    df = self._obj.copy()
    
    df = df.pivot_table(**kwargs)
    
    if index_type == 'flat':
        df = df.fdt.flatten_multiindex(axis='all')
    
    return df

## Series Methods

### Find between text
Extracts number from string

In [ ]:
#export
@pd.api.extensions.register_series_accessor('fdt')
class FastDataSeriesUtilities:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
        
    def find_between_text(self, start_string, end_string):
        series = self._obj
        search_expr = start_string + '(.*)' + end_string
        series = series.str.extract(search_expr)
        return series

/Users/peremolinsayuso/opt/anaconda3/envs/dev_react_jupyterlab/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: registration of accessor <class '__main__.FastDataSeriesUtilities'> under name 'fdt' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  


### extract_number_from_string
Extracts number from string

In [ ]:
#export
@patch_to(FastDataSeriesUtilities)
def extract_number_from_string(self, dtype):
    series = self._obj
    series = series.str.extract('(\d+)')
    series = series.astype(dtype)
    return series

### bin column
Converts numeric column to bin

In [ ]:
#export
@patch_to(FastDataSeriesUtilities)
def bin_column(self, **kwargs):
    series = self._obj
    #Parameters
    p = kwargs
    if p['mode'] == 'size':
        interval_range = pd.interval_range(start=p['start'], freq=p['size'], end=p['end'])
        #print(interval_range)
        series = pd.cut(series, bins=interval_range)
        return series

    if p['mode'] == 'number':
        series = pd.cut(series, bins=p['bin_number'])
        return series

    if p['mode'] == 'quantiles':
        series = pd.qcut(series, q=p['quantiles'])
        return series

    if p['mode'] == 'custom':
        interval_range=pd.IntervalIndex.from_breaks(p['breaks'], closed=p['closed'])
        series = pd.cut(series, bins= interval_range)
        return series

In [ ]:
titanic.Age.max()

80.0

In [ ]:
titanic.Age.min()

0.42

In [ ]:
titanic['age_bin1'] = titanic['Age'].fdt.bin_column(
    mode = 'size',
    size = 20,
    start = 0,
    end = 80 )

In [ ]:
assert type(titanic['age_bin1'].cat.categories.values) == pd.core.arrays.interval.IntervalArray

In [ ]:
titanic['age_bin2'] = titanic['Age'].fdt.bin_column(
    mode = 'number',
    bin_number = 4)

In [ ]:
assert type(titanic['age_bin2'].cat.categories.values) == pd.core.arrays.interval.IntervalArray

In [ ]:
titanic['age_bin3'] = titanic['Age'].fdt.bin_column(
    mode = 'quantiles',
    quantiles = 4)

In [ ]:
assert type(titanic['age_bin3'].cat.categories.values) == pd.core.arrays.interval.IntervalArray

In [ ]:
titanic['age_bin4'] = titanic['Age'].fdt.bin_column(
    mode = 'custom',
    breaks = [5,18,30,50,80],
    closed='right')

In [ ]:
assert type(titanic['age_bin4'].cat.categories.values) == pd.core.arrays.interval.IntervalArray

### fill empty value
Converts numeric column to bin

In [ ]:
#export
@patch_to(FastDataSeriesUtilities)
def fill_empty(self, **kwargs):
    series = self._obj
    p = kwargs
    if p['mode'] == 'function':
        if p['function'] == 'ffill':
            series = series.fillna(method='ffill')
        elif p['function'] == 'bfill':
            series = series.fillna(method='bfill')
        elif p['function'] == 'mean':
            series = series.fillna(series.mean())
        elif p['function'] == 'most_frequent':
            series = series.fillna(series.mode()[0])    
    elif p['mode'] == 'value':
        series = series.fillna(p['value'])
        
    return series

In [ ]:
df_with_holes = pd.DataFrame({'data_with_holes' : [1, 3, 5 , np.NaN, 100, np.NaN, 5]})
df_with_holes

,data_with_holes
0,1.0
1,3.0
2,5.0
3,NaN
4,100.0
5,NaN
6,5.0


In [ ]:
assert df_with_holes['data_with_holes'].fdt.fill_empty(mode='function', function='ffill')[3] == 5

In [ ]:
assert df_with_holes['data_with_holes'].fdt.fill_empty(mode='function', function='bfill')[3] == 100

In [ ]:
assert df_with_holes['data_with_holes'].fdt.fill_empty(mode='function', function='most_frequent')[3] == 5

In [ ]:
df_with_holes['data_with_holes'].mean()

22.8

In [ ]:
assert df_with_holes['data_with_holes'].fdt.fill_empty(mode='function', function='mean')[3] == 22.8

In [ ]:
assert df_with_holes['data_with_holes'].fdt.fill_empty(mode='value', value=13)[3] == 13

### conditional replace
Change a column based on a condition

In [ ]:
#export
@patch_to(FastDataSeriesUtilities)
def replace_based_on_condition(self, cond, when, replace_with=np.NaN):
    series = self._obj
    
    if when == True:
        series = series.mask(cond=cond, other=replace_with)
    elif when == False:
        series = series.where(cond=cond, other=replace_with)

    return series

### Tests

In [ ]:
FIFA_2020 = pd.read_csv(
    sep=",", 
    decimal=".", 
    filepath_or_buffer="../Datasets/players_20.csv")

In [ ]:
FIFA_2020 = FIFA_2020.filter(
    items=["body_type"])

In [ ]:
FIFA_2020["is_standard"] = FIFA_2020.eval("(body_type=='Normal' or body_type=='Stocky' or body_type=='Lean')", engine="python")

In [ ]:
FIFA_2020["corrected"] = FIFA_2020["body_type"].fdt.replace_based_on_condition(
    when=False, 
    cond=FIFA_2020["is_standard"], 
    replace_with="Normal")

TypeError: replace_based_on_condition() got an unexpected keyword argument 'other'

In [ ]:
FIFA_2020["is_not_standard"] = FIFA_2020.eval("~(body_type=='Normal' or body_type=='Stocky' or body_type=='Lean')", engine="python")

In [ ]:
FIFA_2020["corrected2"] = FIFA_2020["body_type"].fdt.replace_based_on_condition(
    when=True, 
    cond=FIFA_2020["is_not_standard"], 
    replace_with="Normal")

In [ ]:
(FIFA_2020["corrected"] == FIFA_2020["corrected2"]).all()

## Exposing methods directly (not used)
This is another approach at exposing pandas methods

In [ ]:
#hide
import pandas as pd
import numpy as np

def flatten_multiindex(self,axis='all'):
            if 'MultiIndex' in str(type(self.columns)) and axis in ('all', 'columns'):
                #print('Removing columns')
                self.columns = self.columns.map(lambda x: '_'.join([str(i) for i in x]))

            if 'MultiIndex' in str(type(self.index)) and axis in ('all', 'index'):
                #print('Removing rows')
                self = self.reset_index()
            return self


@pd.api.extensions.register_dataframe_accessor('flatten_multiindex')
class AccessorMethod(object):
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
    
    def __call__(self, *args, **kwargs):
        return flatten_multiindex(self._obj, *args, **kwargs)

In [ ]:
from nbdev.export import *
notebook2script()